## <font color='663300'>1. Project Title</font>

### Overview of San Diego Coffee Shop Locations and Consumption
#### Yi Li<br>A15536893

## <font color='663300'>2. Questions addressed, why it is important</font>

    Is the coffee shop market in San Diego reaching saturation?
<br>
    The project idea came to me as a result of everyday curiosity and reflection when I noticed <b><font color='0099ff'>Blue Bottle</font></b>, a spectialty coffee shop, opened at UTC this winter quarter (winter 2022). I remembered the endless waiting queue the first day I passed by. And just a week later, the line was gone and I could get my drink soon after ordering. I couldn't help but wondering: where was everybody?
<br>    
<br>
    The project will look into the existing coffee shops in San Diego, showing their locations on map and analyzing a general trend and pattern in terms of the locations of shops. Then I will define features in terms of saturation to find better new shop locations.
<br>    
<br>
Each shop will be analyzed in a tract level. Factors like tract population, annual coffee consumption amount, and access to nearby coffee shops will be used.
<br>    
<br>
I believe this project will cast some insights on San Diego coffee shops and suitable locations to open the next shop for potential investors. It is also a fun project to skim through for coffee lovers and general audiences. 

    

## <font color='663300'> 3. Background and Literature </font>

**1. *The best cities for Coffee Lovers in America*. Apartment Living Tips - Apartment Tips from ApartmentGuide.com.** (2020, December 23). From https://www.apartmentguide.com/blog/best-cities-for-coffee-lovers/ <br>

This article defines and examines 'coffee city' in the U.S. based on coffee shop numbers and per capita (100k people) shop numbers. For example, Berkeley, CA is ranked the first as a coffee city by having ~2000 people sharing a coffee shop on average. It provides me with a benchmark on what a clustered coffee shop area look like in terms of people/coffee shop ratio.


**2. *Coffee shops in San Diego*. LaJolla.com. (2021, March 3).** From https://lajolla.com/article/coffee-shops-san-diego/ <br>
This article talks about the number of coffee shops in San Diego, which 851. This number is greater than the number I obtained from San Diego business listings. It also provides a benchmark for me for further analysis.




**3.*Adapt 2020 Coffee Trends to benefit San Diego: Tri-R Coffee & Vending*. Tri. (2020, August 16).** From https://trircoffee.com/blog/adapt-2020-coffee-trends-benefit-san-diego/<br>

This source provides a general background and justification of the importance of understanding coffee consumption. It says "the majority of adults in American drink coffee and all its related beverages each day, and yet only 63 percent of employees who work outside of their home have access to coffee at work." With that being said, finding suitable locations for coffee shops remains a wise move.



**4. Fruhlinger, J. (2019, November 20). *22,842 coffee shops of America: An interactive map.* The Business of Business.** From https://www.businessofbusiness.com/articles/coffee-shops-of-america-an-interactive-map/

This map shows a map of coffee shops and a way of visualizing data.



**5. *U.S. coffee consumers now expect an elevated coffee experience at and away from home.* The NPD Group.** From https://www.npd.com/news/press-releases/2021/u-s-coffee-consumers-now-expect-an-elevated-coffee-experience-at-and-away-from-home/ 

The article talks about coffee consumption during pandemic -- "the split between at-home and away-from-home coffee consumption became 81% from home and 19% from foodservice during the pandemic." With a majority consuming in-home coffee, the average San Diego in-home coffee conusumption data obtained from Esri 2021 Consumer Dataset can well represent the level of coffee consumption in a tract.

## <font color='663300'>4. Libraries and ArcGIS modules</font>

1. GeoAccessor -- this module is imported because some datasets were initially of pandas dataframe type or Geopandas dataframe type. It is used to transform those to spatially enabled dataframes.<br>
2. Geoenrichment -- this module is essential to the project, as it provides critical information, like in-home coffee consumption, population of tracts, daytime population, etc.
3. Polygon from Geometry -- Polygon is a new import function comparing to initial idea. It is used along with geoenrichment, so that each tract SHAPE can be enriched.
4. Geoencoding -- For datasets that do not have geographical information, geocoding helps a lot in finding their longitude and latitude. For similar name locations, geocoding helps remove duplicates if having the same geographical information.
5. Webmap is used to show the processed map inline.

In [3]:
import arcgis
from arcgis.gis import GIS
from IPython.display import display
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd
import geopandas as gpd
from arcgis.geoenrichment import *
from arcgis.geometry import Polygon
from arcgis.geocoding import Geocoder, get_geocoders, geocode
from arcgis.mapping import WebMap

In [4]:
import getpass
username=input('Enter username')
password=getpass.getpass('Enter password: ')
gis = GIS(username=username,password=password)

Enter usernamedsc170wi22_592
Enter password: ········


## <font color='663300'> 5. Data Sources</font>

1. The coffee_shop layer (https://ucsdonline.maps.arcgis.com/home/item.html?id=d304be60d5ea4afd8042a3a93f6f8b44#overview) records coffee shop points, locations, names, and which tract they belong to, etc.  This source comes from both of San Diego business listings (https://www.sandiego.gov/treasurer/taxesfees/btax/nblactive) and scraped data from Google Maps （searching keywords are "coffee shop San Diego") using PhantomBuster （https://phantombuster.com ）
<br>
<br>
2. Tracts information is a local file downloaded from https://www.sandag.org/index.asp?subclassid=52&fuseaction=home.subclasshome. It contains the geographical info of each San Diego tract.
<br>
<br>
3. Other data source include geoenrichment with Esri 2021 Demographics dataset(https://doc.arcgis.com/en/esri-demographics/data/updated-demographics.htm) and Esri 2021 Consumer Spending (https://doc.arcgis.com/en/esri-demographics/data/consumer-spending.htm). Each contain useful variables in a tract level, such as population, daytime populationm and in-home coffee consumption.
<br>
<br>
4. Other layers, like driving buffers, and market saturation level layer are processed dataset using above data sources.
<br>
<br>
There's no single feature to decide if a place is reaching a saturated coffee market. In fact, analyzing coffee shop location patterns in tract level may overlook the real geographical differences already. Also, there is no tract-level data in terms of coffee consumption away from home, therefore in-home coffee consumption is used here to reflect such idea.

In [10]:
# these are all processed dataframes using original dataframes
tract_coffee = gis.content.get('647ae4e0a1ea4414b093d29c3c5935b5').layers[0].query().sdf
coffee_shop  = gis.content.get('d304be60d5ea4afd8042a3a93f6f8b44').layers[0].query().sdf
tracts = gis.content.get('4d9ab58add054431b4c36fcd76583313').layers[0].query().sdf
buffer_3_min_drive = gis.content.get('ea9a4c58b8474f86a71f53eede96555e').layers[0].query().sdf
buffer_5_min_drive = gis.content.get('ab1dfe23b8314286830d25dde1ac03a6').layers[0].query().sdf
saturation_level = gis.content.get('973fbf3766bf449d990af7648593dee0').layers[0].query().sdf

## <font color='663300'>6. Data Cleaning</font>

#### The following cells are to generate a clean coffee shop dataset using business listings.

In [6]:
# business A-K
temp1 = pd.read_csv('business_1.csv')

# business L-Z
temp2 = pd.read_csv('business_2.csv')
business = pd.concat([temp1,temp2])
business.head(2)

,BUSINESS ACCT#,DBA NAME,OWNERSHIP TYPE,ADDRESS,CITY,ZIP,STATE,BUSINESS PHONE,OWNER NAME,CREATION DT,START DT,EXP DT,NAICS,ACTIVITY DESC
0,2020002344,#0205,LLC,4704 IMPERIAL AVE,SAN DIEGO,92113-5001,CA,(619) 436-5006,CF UNITED CVX HOLDINGS LLC,2/7/2020,11/14/2019,11/30/2022,44711,GASOLINE STATIONS WITH CONVENIENCE STORES
1,2019021709,#1 FIFTH AVE,LLC,3845 05TH AVE,SAN DIEGO,92103-3140,CA,(619) 299-1911,ICONIC BAR INVESTMENTS LLC,10/15/2019,10/11/2019,10/31/2022,72241,DRINKING PLACES (ALCOHOLIC BEVERAGES)


##### <font color='660066'>since the dataset contains all business categories, I browsed the dataset in excel, hand-picked keywords related to coffee shops, and searched online to filter businesses using NAICS code (722/445) </font>

In [7]:
# food related NAICS code starts with 722 (food services and drinking places) 
#                                  or 445 (Food and Beverage Stores)
# lookup https://www.naics.com/code-search/?naicstrms=coffee

coffee_keywords = 'COFFEE|CAFE|STARBUCKS|ESPRESSO|DUNKIN|BASKIN'
related_cols = ['DBA NAME','ADDRESS','CREATION DT']

food = business[(business['NAICS'].astype(str).str.startswith('722'))|business['NAICS'].astype(str).str.startswith('445')]
coffee = food[(food['DBA NAME'].str.contains(coffee_keywords)==True)|(food['OWNER NAME'].str.contains(coffee_keywords)==True)]#.reset_index(drop=True)[related_cols]
coffee.head(2) #there are 317 in total

,BUSINESS ACCT#,DBA NAME,OWNERSHIP TYPE,ADDRESS,CITY,ZIP,STATE,BUSINESS PHONE,OWNER NAME,CREATION DT,START DT,EXP DT,NAICS,ACTIVITY DESC
336,2016003500,85 C BAKERY CAFE,LLC,8265 MIRA MESA BLVD,SAN DIEGO,92126-2603,CA,(714) 459-9585,WINPIN 85 INVESTMENTS LLC,3/17/2016,5/29/2016,5/31/2022,722,FOOD SERVICES & DRINKING PLACES
339,2018018106,85C BAKERY CAFE,LLC,4313 LA JOLLA VILLAGE DR SUITE 2275,SAN DIEGO,92122-1475,CA,NaN,WINPIN 85 INVESTMENTS LLC,8/21/2018,8/17/2018,8/31/2022,722211,LIMITED-SERVICE RESTAURANTS


##### <font color='660066'>geoencode coffee shops with locations</font>

In [22]:
# geocode the 317 coffee shop with long/lat
long = []
lat = []
for i in range(coffee.shape[0]):
    add = geocode(coffee['DBA NAME'][i]+',' + coffee['ADDRESS'][i]+',San Diego,California')[0]['location']
    long.append(add['x'])
    lat.append(add['y'])

In [23]:
coffee['longitude'] = long
coffee['latitude'] = lat
coffee.head(2)

,DBA NAME,ADDRESS,CREATION DT,longitude,latitude
0,85 C BAKERY CAFE,8265 MIRA MESA BLVD,3/17/2016,-117.145338,32.912607
1,85C BAKERY CAFE,4313 LA JOLLA VILLAGE DR SUITE 2275,8/21/2018,-117.210098,32.871599


##### <font color='660066'>Another major source of coffee shops comes from google map. <br>The scraped_coffee dataset below has a different format than the business listing dataset. <br><br>It is also noted that those 2 dataframes may have overlaps. Therefore, I geoencoded both using detailed address information, merged them, and removed duplicates. </font>

In [24]:
# scraped data from google maps -- I checked that there are overlaps as well as not covered ones
related_cols = ['title','address']
scraped_coffee = pd.read_csv('final_all_coffee.csv')[related_cols]
scraped_coffee.head()

,title,address
0,James Coffee Co.,"2355 India St, San Diego, CA 92101, United States"
1,Brickyard Coffee & Tea,"675 W G St, San Diego, CA 92101, United States"
2,Achilles Coffee Roasters on Cortez Hill,"703 Ash St, San Diego, CA 92101, United States"
3,Holsem Coffee,"2911 University Ave, San Diego, CA 92104, Unit..."
4,Achilles Coffee Roasters @ The Rey,"800 B St, San Diego, CA 92101, United States"


In [26]:
# scraped coffee shop information from 

long = []
lat = []
for i in range(scraped_coffee.shape[0]):
    add = geocode(str(scraped_coffee.title[i]) + ',' + str(scraped_coffee.address[i]))[0]['location']
    long.append(add['x'])
    lat.append(add['y'])

scraped_coffee['longitude'] = long
scraped_coffee['latitude'] = lat
scraped_coffee['creation dt'] = 'unknown'
scraped_coffee.head(2)

,title,address,longitude,latitude,creation dt
0,James Coffee Co.,"2355 India St, San Diego, CA 92101, United States",-117.170353,32.729117,unknown
1,Brickyard Coffee & Tea,"675 W G St, San Diego, CA 92101, United States",-117.168834,32.712331,unknown


In [27]:
# merge 2 coffee shop dataframes
col1 = ['name','address','creation_date','long','lat']
col2 = ['name','address','long','lat','creation_date']
coffee.columns = col1
scraped_coffee.columns = col2
merged_coffee = coffee.merge(scraped_coffee, left_on=['long','lat'], right_on=['long','lat'], how='outer')
merged_coffee.head()

,name_x,address_x,creation_date_x,long,lat,name_y,address_y,creation_date_y
0,85 C BAKERY CAFE,8265 MIRA MESA BLVD,3/17/2016,-117.145338,32.912607,NaN,NaN,NaN
1,85C BAKERY CAFE,4313 LA JOLLA VILLAGE DR SUITE 2275,8/21/2018,-117.210098,32.871599,NaN,NaN,NaN
2,85C BAKERY CAFE,5575 BALBOA AVE SUITE 330,10/9/2014,-117.180327,32.819734,85C Bakery Cafe - Balboa Mesa,"5575 Balboa Ave Suite 330, San Diego, CA 92111...",unknown
3,ACENTO COFFEE ROASTERS LLC,5334 BANKS ST,8/21/2020,-117.198818,32.764274,Acento Coffee Roasters,"5334 Banks St, San Diego, CA 92110, United States",unknown
4,ACHILLES COFFEE ROASTERS,100 PARK PLAZA SUITE 175,3/11/2019,-117.155211,32.706970,Achilles Coffee Roasters @ Park 12,"100 Park Plz Suite 175, San Diego, CA 92101, U...",unknown


In [ ]:
# merge the two datasets together based on location
name = []
address = []
creation_date = []

for i in range(merged_coffee.shape[0]):
    if str(merged_coffee.name_x[i])=='nan':
        name.append(merged_coffee.name_y[i])
    else:
        name.append(merged_coffee.name_x[i])
    
    if str(merged_coffee.address_x[i])=='nan':
        address.append(merged_coffee.address_y[i])
    else:
        address.append(merged_coffee.address_x[i])
    
    if str(merged_coffee.creation_date_x[i])=='nan':
        creation_date.append(merged_coffee.creation_date_y[i])
    else:
        creation_date.append(merged_coffee.creation_date_x[i])
coffee_shop = pd.DataFrame({'name':name,
             'address':address,
             'creation_date':creation_date,
             'long':merged_coffee.long,
             'lat':merged_coffee.lat})

# 571 coffee shopes in total

In [57]:
coffee_shop = pd.DataFrame.spatial.from_xy(coffee_shop,x_column = 'long', y_column='lat')
coffee_shop = gpd.GeoDataFrame(coffee_shop, geometry='SHAPE', crs="epsg:4326")
coffee_shop.head(3)

,FID,fid_1,name,address,creation_d,long,lat,opening_ye,years_till,tract_id,tract_pop,SHAPE
0,1,501,Superbloom Coffee & Juice,"2688 E Mission Bay Dr, San Diego, CA 92109, Un...",unknown,-117.209129,32.789920,NaN,NaN,76.00,3973.0,"{""x"": -13047660.503386391, ""y"": 3867452.465207..."
1,2,502,The Mission Bay Beach Club,"2688 E Mission Bay Dr, San Diego, CA 92109, Un...",unknown,-117.209129,32.789920,NaN,NaN,76.00,3973.0,"{""x"": -13047660.503386391, ""y"": 3867452.465207..."
2,3,503,Crown Point Coffee,"3510 Ingraham St # 101, San Diego, CA 92109, U...",unknown,-117.236840,32.783091,NaN,NaN,77.01,3673.0,"{""x"": -13050745.331012484, ""y"": 3866548.186629..."


##### <font color='660066'>The resulting dataset coffee_shop comes from two sources mentioned above, while with one source having opening_year information and the other not. I imputed those empty values in the second data source with np.nan/unknown. It was planned to be used initially, while later I discarded this idea since there were not enough information. </font>

In [ ]:
coffee_shop['opening_year'] = np.nan
coffee_shop['years_till_2022'] = np.nan
for i in range(coffee_shop.shape[0]):
    if coffee_shop.creation_date[i]!='unknown':
        coffee_shop.opening_year[i] = int(str(coffee_shop.creation_date[i])[-4:])
        coffee_shop.years_till_2022[i] = 2022-int(str(coffee_shop.creation_date[i])[-4:])

#### The tract dataset records 627 San Diego tracts and their locations, I would like to populate them with population, daytime population, coffee consumption, etc.

In [61]:
useful_variables = ['X1122_X', #coffee
                    'X1122_A', #coffee average
                   'X1119_X',  #Nonalcoholic Beverages
                    'X1119_A', #Nonalcoholic Beverages: Average
                ]
cols = ['coffee','coffee_avg','nonalcohol_beverage','nonalchohol_beverage_avg']

coffee = []
coffee_avg = []
nonalcohol_beverage = []
nonalchohol_beverage_avg = []
population = []
daytime_pop =[]

for i in range(tracts.shape[0]):
    # ACSTOTPOP
    pop = enrich(study_areas=[Polygon(tract_coffee.SHAPE[i])],analysis_variables=['TOTPOP_CY'])['TOTPOP_CY'][0]
    day_pop =  enrich(study_areas=[Polygon(tract_coffee.SHAPE[i])],analysis_variables=["DPOP_CY"])['DPOP_CY'][0]
    enriched = enrich(study_areas=[Polygon(tracts.SHAPE[i])],data_collections=['Food'])
    coffee.append(enriched.X1122_X[0])
    coffee_avg.append(enriched.X1122_A[0])
    nonalcohol_beverage.append(enriched.X1119_X[0])
    nonalchohol_beverage_avg.append(enriched.X1119_A[0])
    population.append(pop)
    daytime_pop.append(day_pop)

In [58]:
tracts['coffee'] = coffee
tracts['coffee_avg'] = coffee_avg
tracts['nonalcohol_beverage'] = nonalcohol_beverage
tracts['nonalchohol_beverage_avg'] = nonalchohol_beverage_avg
tracts['population'] = population
tracts['daytime_po'] = daytime_pop
#tract_coffee_fl = tracts.spatial.to_featurelayer(title ='tract_coffee_info', gis=gis, tags="final project")
#tract_coffee = tract_coffee_fl.layers[0].query().sdf
tract_coffee.head(3)

,FID,fid_1,fid_12,fid_123,fid_1234,tract,shape_area,shape_len,shape_ar_1,shape_leng,...,SHAPE,within_drive_buffer,above_5000_pop,above_5000_daytime_pop,above_avg_consumption,saturated,saturated_daytime,potential,potential_daytime,level
0,1,1,1,1,1,15.0,8.693887e+06,12443.272111,1.144882e+06,4515.214375,...,"{""rings"": [[[-13038003.2610141, 3861991.956066...",True,False,False,True,1,1,0,0,3
1,2,2,2,2,2,16.0,7.407379e+06,11329.616060,9.756497e+05,4110.164970,...,"{""rings"": [[[-13038360.258501, 3862867.4024732...",True,True,False,False,0,0,0,0,2
2,3,3,3,3,3,17.0,6.714940e+06,10791.678584,8.845680e+05,3914.594032,...,"{""rings"": [[[-13038315.3966399, 3863622.185421...",True,False,False,False,0,0,0,0,2


In [65]:
coffee_shop.head(2)

,FID,name,address,creation_d,long,lat,opening_ye,years_till,SHAPE
0,1,85 C BAKERY CAFE,8265 MIRA MESA BLVD,3/17/2016,-117.145338,32.912607,2016.0,6.0,"{""x"": -13040559.333636776, ""y"": 3883709.678001..."
1,2,85C BAKERY CAFE,4313 LA JOLLA VILLAGE DR SUITE 2275,8/21/2018,-117.210098,32.871599,2018.0,4.0,"{""x"": -13047768.422169274, ""y"": 3878273.251595..."


In [69]:
tract_coffee.head(2)

,FID,fid_1,tract,shape_area,shape_len,shape_ar_1,shape_leng,coffee,coffee_avg,nonalcohol,nonalchoho,population,Shape__Area,Shape__Length,SHAPE
0,1,1,15.0,8.693887e+06,12443.272111,1.144882e+06,4515.214375,274683,142.62,1257390,652.85,3827,1.144882e+06,4515.214375,"{""rings"": [[[-13038003.2610141, 3861991.956066..."
1,2,2,16.0,7.407379e+06,11329.616060,9.756497e+05,4110.164970,171932,74.98,845893,368.90,5821,9.756498e+05,4110.164970,"{""rings"": [[[-13038360.258501, 3862867.4024732..."


##### <font color='660066'>Now assign each coffee shop with its corresponding tract number</font>

In [6]:
tract_id = [None] * coffee_shop.shape[0]
tract_pop = [None] * coffee_shop.shape[0]
for i in range(coffee_shop.shape[0]):
    for j in range(tract_coffee.shape[0]):
        if tract_coffee.SHAPE[j].contains(coffee_shop.SHAPE[i]):
            tract_id[i] = tract_coffee.tract[j]
            tract_pop[i] = tract_coffee.population[j]
coffee_shop['tract_id'] = tract_id
coffee_shop['tract_pop'] = tract_pop
coffee_shop.head(2)

,FID,name,address,creation_d,long,lat,opening_ye,years_till,SHAPE,tract_id,tract_pop
0,1,85 C BAKERY CAFE,8265 MIRA MESA BLVD,3/17/2016,-117.145338,32.912607,2016.0,6.0,"{""x"": -13040559.333636776, ""y"": 3883709.678001...",83.51,4225.0
1,2,85C BAKERY CAFE,4313 LA JOLLA VILLAGE DR SUITE 2275,8/21/2018,-117.210098,32.871599,2018.0,4.0,"{""x"": -13047768.422169274, ""y"": 3878273.251595...",83.40,9762.0


##### <font color='660066'>the top 10 tracts are tracts with the most coffee shops; and the last 10 tracts are tracts with the fewest coffee shops</font>

In [43]:
top_10_tracts = coffee_shop.groupby('tract_id',as_index=False)[['name']].count().sort_values('name',ascending=False).reset_index()[0:10].rename(columns={'name':'count_shops'})
top_10_tracts = top_10_tracts.merge(tract_coffee, left_on='tract_id',right_on='tract')[['tract_id','count_shops','coffee_avg','population','SHAPE']]
last_10_tracts = coffee_shop.groupby('tract_id',as_index=False)[['name']].count().sort_values('name').reset_index()[0:10].rename(columns={'name':'count_shops'})
last_10_tracts = last_10_tracts.merge(tract_coffee, left_on='tract_id',right_on='tract')[['tract_id','count_shops','coffee_avg','population','SHAPE']]
top_10_tracts['people_shop_ratio'] = top_10_tracts.population/top_10_tracts.count_shops
last_10_tracts['people_shop_ratio'] = last_10_tracts.population/last_10_tracts.count_shops
top_10_tracts['top'] = 1
last_10_tracts['top'] = 0
merged = pd.concat([top_10_tracts,last_10_tracts])
merged.head()

,tract_id,count_shops,coffee_avg,population,SHAPE,people_shop_ratio,top
0,53.00,36,124.86,6945,"{'rings': [[[-13042599.9726413, 3858182.793970...",192.916667,1
1,54.00,33,179.80,8674,"{'rings': [[[-13043397.4652264, 3858180.544737...",262.848485,1
2,58.00,19,172.93,4928,"{'rings': [[[-13043113.0441787, 3859245.196965...",259.368421,1
3,85.11,19,134.57,5097,"{'rings': [[[-13041641.4004879, 3873713.014615...",268.263158,1
4,93.04,13,127.03,8661,"{'rings': [[[-13037144.8721413, 3866981.435911...",666.230769,1


## <font color='663300'>7. Descriptive statistics for the data</font>

presentation StoryMap: https://storymaps.arcgis.com/stories/797a746683bb4c2ab7f01f6a18589e83

#### <font color='b35900'> sd_avg provides a benchmark for the entire area</font>

In [17]:
# sd average
useful_variables = ['X1122_X', #coffee
                    'X1122_A', #coffee average
                   'X1119_X',  #Nonalcoholic Beverages
                    'X1119_A', #Nonalcoholic Beverages: Average
                ]
cols = ['coffee','coffee_avg','nonalcohol_beverage','nonalchohol_beverage_avg']

sd_avg = enrich(study_areas=['San Diego, CA'],data_collections=['Food'])[useful_variables]
sd_avg.columns = cols

sd_avg

,coffee,coffee_avg,nonalcohol_beverage,nonalchohol_beverage_avg
0,3565150,133.3,16415224,613.77


#### <font color='b35900'>there are 571 coffee shops in 175 tracts</font>

In [15]:
print(str(coffee_shop.shape[0])+' coffee shops')
print(str(coffee_shop.tract_id.unique().size)+' tracts')

571 coffee shops
175 tracts


#### <font color='b35900'>the average in-home coffee consumption in San Diego is $133.3</font>

In [18]:
print(sd_avg.coffee_avg[0])

133.3


#### <font color='b35900'>the bench mark for population is 5000 because of the average tract population</font>

In [19]:
tract_coffee.population.mean()

5180.692185007974

#### <font color='b35900'>The map below shows the locations of coffee shops in San Diego. With each orange point as a coffee shop, the heatmap layer below shows how clustered they are. It can be seen that clusters of coffee shops are by the coast in the south of San Diego. </font>

In [13]:
coffee_shop_overview = gis.content.get('b1408cad83f64f55a334321787238c97')
WebMap(coffee_shop_overview)

MapView(hide_mode_switch=True, layout=Layout(height='400px', width='100%'))

With more coffee shops clustered by the coast, there could be spatial correlation because of underlying similarities, like dense population and flat ground so that more is located there. In the Analysis section, population factor will be examined by per capita value.

#### <font color='b35900'>The map below shows a 3-min driving buffer (purple) and a 5-min driving buffer (pink). Each orange point is a coffee shop. It builds a general sense of accessbility of coffee shops in San Diego. While it covers quite an expanding area, there are also places that are neither natural reserve nor covered. </font>

In [14]:
driving_buffer_overview = gis.content.get('4614e7433a134a3bbbbc0fc7e1493f3e')
WebMap(driving_buffer_overview)

MapView(hide_mode_switch=True, layout=Layout(height='400px', width='100%'))

## <font color='663300'>8. Analysis</font>

#### <font color='2d8659'>general outline</font>

1. The below cells (first map) shows coffee shop clusters by normalizing population. With denser tracts by the coast, it can be assumed that outside factors may contribute to this phenomeon.
<br>
2. Then I'd like to categorize each tract to a saturation level. The 3 criteria are stated below. The logic is that, an area with saturated coffee shop market will be covered by nearby coffee shop buffers, and have above average in-home coffee consumption. This could have to small tracts or larger tracts, and opening a new coffee shop there doesn't seem like a good idea. The assigned value for saturated tract is 3.
3. The tracts that have a saturation level 1 are seen as potential destinations for opening a new coffee shop. They are defined as having population/daytime population over 5000 (San Diego average) so there are more customers nearby; not covered (at least a part of the tract) by nearby coffee shop buffers; and last, the average in-home coffee consumption has not reached the average.
4. All other tracts are categorized in level 2.
<br> 
<br>The finding will be discussed in the last section

**saturation criteria:**

1. areas are within in the 5-min driving buffer of exisiting coffee shops
2. the average coffee consumption is above San Diego average

**potential for growth criteria**

1. population > 5000
2. areas are NOT within in the 5-min driving buffer of exisiting coffee shops
3. the average coffee consumption is BELOW San Diego average

#### <font color='2d8659'>The below map shows the processed coffee shop clustering per capita, which is a normalized value. In San Diego, the top 10 tracts have a really high coffee shop numbers. As a reference, with per capita (100k) 46.9 coffee shops, Berkeley is ranked 1st as a coffee city, according to  ApartmentGuide. That's ~2000 people per coffee shop.</font>
<br>
The top 10 tracts (colored in yellow) are clustered by the coast. The people-coffee-shop ratio can be as small as ~250 -- meaning 250 people per coffee shop in that tract. The last 10 tracts are colored in blue, and the people-coffee-shop ratio is really high -- range from ~5000 and above, indicating a lack of coffee shops in those areas.

In [15]:
top_last_tracts_overview = gis.content.get('a54bf61fde90432a975606e11fc05976')
WebMap(top_last_tracts_overview)

MapView(hide_mode_switch=True, layout=Layout(height='400px', width='100%'))

#### <font color='2d8659'>saturated</font>

In [46]:
# The two dataset are processed so that each tract will be assigned a saturation level based on criteria. 
# Tracts are divided based on their saturation levels.

within_drive_buffer = [] # whether this tract is accessible to coffee shops by driving buffer
above_5000_pop = [] # whether this tract has over 5000 people
above_5000_daytime_pop = [] # or if it exceeds 5000 daytime population
above_avg_consumption = [] # whether this tract exceeds San Diego average in-home coffee consumption

for i in range(tract_coffee.shape[0]):
    if i == 152 or i == 251:
        within_drive_buffer.append(False)
    else:
        within_drive_buffer.append(buffer_5_min_drive.SHAPE[0].contains(tract_coffee.SHAPE[i]))
    above_5000_pop.append(tract_coffee.population[i] > 5000)
    above_5000_daytime_pop.append(tract_coffee.daytime_po[i] > 5000)
    above_avg_consumption.append(tract_coffee.coffee_avg[i] > sd_avg.coffee_avg[0])
    
tract_coffee['within_drive_buffer'] = within_drive_buffer
tract_coffee['above_5000_pop'] = above_5000_pop
tract_coffee['above_5000_daytime_pop'] = above_5000_daytime_pop
tract_coffee['above_avg_consumption'] = above_avg_consumption

saturated = (tract_coffee.within_drive_buffer * above_avg_consumption)
saturated_cnt = saturated.sum()
saturated_daytime = (tract_coffee.within_drive_buffer * above_avg_consumption)
saturated_daytime_cnt = saturated_daytime.sum()
potential = ((tract_coffee.within_drive_buffer==False)*(tract_coffee.above_5000_pop)*(tract_coffee.above_avg_consumption==False))
potential_cnt = potential.sum()
potential_daytime = ((tract_coffee.within_drive_buffer==False)*(tract_coffee.above_5000_daytime_pop)*(tract_coffee.above_avg_consumption==False))
potential_daytime_cnt = potential_daytime.sum()
tract_coffee['saturated'] = saturated.astype(int)
tract_coffee['saturated_daytime'] = saturated_daytime.astype(int)
tract_coffee['potential'] = potential.astype(int)
tract_coffee['potential_daytime'] = potential_daytime.astype(int)

#--------------assign saturation level
level = []
for i in range(tract_coffee.shape[0]):
    if (int(tract_coffee.saturated[i]+ tract_coffee.saturated_daytime[i]) > 0):
        level.append(int(3))
    elif (int(tract_coffee.potential[i] + tract_coffee.potential_daytime[i]) > 0):
        level.append(int(1))
    else:
        level.append(int(2))
tract_coffee['level'] = level

In [48]:
final = tract_coffee.loc[:,['SHAPE','tract','coffee_avg','level','population','daytime_po','within_drive_buffer','level']]
final.within_drive_buffer = final.within_drive_buffer.astype(int)
final.head(3)

,SHAPE,tract,coffee_avg,level,population,daytime_po,within_drive_buffer,level
0,"{""rings"": [[[-13038003.2610141, 3861991.956066...",15.0,142.62,3,3998,3059,1,3
1,"{""rings"": [[[-13038360.258501, 3862867.4024732...",16.0,74.98,2,5728,4355,1,2
2,"{""rings"": [[[-13038315.3966399, 3863622.185421...",17.0,97.32,2,4963,3431,1,2


In [53]:
#saturation_level_fl = final.spatial.to_featurelayer(title ='saturation_level_final', gis=gis, tags="final project")

## <font color='663300'>9. Summary of products and results</font>

In [49]:
print('there are ' + str(saturated_cnt) + ' tracts that are considered to have saturated coffee shop market')

there are 65 tracts that are considered to have saturated coffee shop market


In [56]:
print('there are ' + str(potential_cnt) + ' tracts that are considered to have unsarturated coffee shop market, therefore more growth potential if opening a new store')

there are 91 tracts that are considered to have unsarturated coffee shop market, therefore more growth potential if opening a new store


The map below shows saturation levels of each tract.The dark brown indicates a saturation level of 3, meaning a saturated coffee shop market based on the definition. The lighter gray indicates suitable locations for new coffee shops, and the light brown is in between. Though not a lot of coffee shops are located in the vast area in the east, there shows no messages in urging people to open a new shop there. Most saturated tracts are by the coast and in a close cluster. 
<br><br>Back to the question asked in the beginning: 'Is the coffee shop market in San Diego reaching saturation?' My answer would be, while there are a number of tracts having potential for growth, there are definitely saturated areas, that show a clear pattern of being clusterd by the coast and nearby each other.

In [16]:
saturation = gis.content.get('701b6626e8614a49a403e5c79a912e8f')
WebMap(saturation)

MapView(hide_mode_switch=True, layout=Layout(height='400px', width='100%'))

## <font color='663300'>10. Discussion</font>

The finding provides a new angle of understanding coffee shops in San Diego. The recommended coffee shops mentioned in the sources are in fact clustered in a manner that maybe further interpreted as some sort of "coffee culture." This finding also provides an explanation for my observation that the waiting queue was gone soon after Blue Bottle opened -- I zoomed in the map and found it located in a saturated tract. So it could be understood that people have a lot of other places to get their daily coffee.

The 3 min/5 min driving buffers are picked upon everyday experiences, therefore it may not be an ideal metric to measure accessiblity to coffee. Also, because all the data is analyzed in a tract level, it's possible that the factors contribute to clustering difference is removed in that level.

Meanwhile, the fact that lacking an away-home coffee consumption data casts doubts to the representativeness of in-home coffee consumption data. A high in-home coffee consumption may lead to lower away-home coffee consumption because coffee could be prepared indoor; it also means greater interest in consuming coffee, since they need to leave for work, which in-home coffee cannot fulfill their needs.

## <font color='663300'>11. Conclusions and future work</font>

I think I have managed to answer most of my initial question. While completing the project, there were a lot of things that I think can be incorporated to better and more logically answer the question. The topic transfers from 'deciding if San Diego has a saturated coffee shop market' to 'which tracts in San Diego has reached a saturated state by definition,' to eventually 'where are suitable places to open a new coffee shop.' To better address those, a network analysis in terms of traffic, population, surroundings can be used. Also, not all coffee shops listed above are of the top tier -- we would like to find locations that will maximize return of profit, therefore more factors need to be considered if making a cohesive conclusion. 

Based on what we have now, a visualization of new coffee shops each year can be achieved if exists such dataset. This helps scrutinize the trend of number of newly opened coffee shops each year -- are there more and more new shops opening in this area or fewer by years -- and the trend of closing coffee shops each year. The total number of coffee shops each year can be helpful in evaluating the growth. Last but not least, I think the project can be of some help for general coffee lovers, shop explorers, and potential coffee oweners to understand a little bit more about coffee shops in San Diego. 